## Cordic算法

### 算法简介
1971年， Walther提出Cordic统一算法  
算法公式：
$$
\begin{bmatrix}
 x(n)\\y(n)

\end{bmatrix}=\prod_{i=1}^{n}\cos (m^{1/2}a_{i} ) \begin{bmatrix}
 1 & -m^{1/2}\sigma _{i}\tan (m^{1/2}a_i)\\m^{1/2}\sigma _{i}\tan (m^{1/2}a_i)
  &1
\end{bmatrix}\begin{bmatrix}
 x(0)\\y(0)

\end{bmatrix}$$


$$Z(n)=Z(0)+\sum_{i=0}^{n}\sigma  _ia_i$$

$$------运算--------$$
$$圆周运算： m = 1$$
$$双曲旋转运算： m = -1$$
$$线形旋转运算: m = 0 $$
$$------模式--------$$
$$ 旋转模式：结果使Z(n)=0 $$ 
$$ 向量模式：结果使y(n)=0 $$

$$\sigma _i=\left\{\begin{matrix}\ \ \ \ sign(Z(i))\ \  in\ rotation\  mode\\sign(y(i))\ \  in\ vector\ mode
\end{matrix}\right.$$

常用微转角$a_i$旋转方案为  
$$tan(a{_i})=2^{-i}\ \ m=1$$ 
$$tanh(a{_i})=2^{-i}\ \ m=-1$$ 
$$a{_i}=2^{-i}\ \ m=1$$

迭代运算为：  
$$X_{i+1}=X_i-m\times \sigma _i\times Y_i \times 2^{-i}$$
$$Y_{i+1}=Y_i+\sigma _i\times X_i \times 2^{-i}$$
$$Z_{i+1}=Z_i-\sigma _i \times a_i$$

在所有级旋转之后进行一次模校正运算，模校正因子：
$$k_{m,n}=\prod_{i=1}^{n}cos(m^{1/2}a_i) $$

### 公式中的一些提前可算参数以及旋转模式下计算$\cos$函数
旋转模式：  
初始值：$x_0=K$ ; $Y_0=0$ ;    $Z_n=\theta$  
输出：$x_n=\cos(\theta)$   ;  $Y_n=\sin(\theta)$  ;   $Z_n=0$

1. 微转角$a_i$：$$tan(a{_i})=2^{-i}\ \ m=1$$ 

In [41]:
import numpy as np
def cal_a_i(n): #计算n次迭代之后的旋转角数组
    a_i = []
    for i in range(n):
        a_i.append(np.arctan(pow(2,-i)))
    return a_i  
a_i_test = cal_a_i(15)
print(a_i_test)

[0.7853981633974483, 0.4636476090008061, 0.24497866312686414, 0.12435499454676144, 0.06241880999595735, 0.031239833430268277, 0.015623728620476831, 0.007812341060101111, 0.0039062301319669718, 0.0019531225164788188, 0.0009765621895593195, 0.0004882812111948983, 0.00024414062014936177, 0.00012207031189367021, 6.103515617420877e-05]


2. 模校正因子$k_{m,n}$
$$k_{m,n}=\prod_{i=1}^{n}cos(m^{1/2}a_i) $$

In [42]:
def cal_k(m, n, a_i):
    k = []
    for i in range(n):
        if len(k) == 0:
            k.append(np.cos(pow(m, 1/2)*a_i[0]))
        else:
            k.append(k[-1]*np.cos(pow(m, 1/2)*a_i[i]))
    return k
k_test = cal_k(1, 10, a_i_test)  #旋转模式
print(k_test)

[0.7071067811865476, 0.6324555320336759, 0.6135719910778964, 0.6088339125177524, 0.6076482562561683, 0.607351770141296, 0.6072776440935261, 0.6072591122988928, 0.6072544793325625, 0.6072533210898753]


### Python算法实现参考

_将Cordic库加入路径_  
_Cordic_ 库来源于 https://people.sc.fsu.edu/~jburkardt/py_src/cordic/cordic.py
具体代码:[Cordic_Python计算](./src/Cordic.py)

In [43]:
import sys
import numpy as np
sys.path.append('./src')
import Cordic

1. 将输入rad制的角度 归一化  
**angle_shift $(\alpha, \beta)$**shifts angle $\alpha$ to lie between $\beta$ and $\beta+2\pi$.(the result is $\gamma$)  其他函数先调用了这个函数  
The resulting angle $\gamma$ has all the same trigonometric function values as $\alpha$.

$$\cos((\beta-\alpha) \bmod 2*\pi) = \cos(\beta-\alpha)$$
$$\cos(\gamma) = cos(\alpha) $$

$if \alpha<\beta:$  
$ \gamma = \beta - ((\beta-\alpha) \bmod 2*\pi)+2\pi$  
$else:$  
$ \gamma = \beta + ((\alpha-\beta) \bmod 2*\pi)$

In [44]:
Cordic.angle_shift_test()


angle_shift_test:
  angle_shift() shifts angle ALPHA to lie between
  BETA and BETA+2 PI.

           ALPHA          BETA   ALPHA_SHIFT     BETA+2 PI

     -7.987035      6.090034     10.862521     12.373220
     -7.972752      6.090034     10.876804     12.373220
     -4.340241      6.090034      8.226130     12.373220
      9.184636      6.090034      9.184636     12.373220
      1.102044      6.090034      7.385230     12.373220
      3.244779      6.090034      9.527964     12.373220
      4.883249      6.090034     11.166434     12.373220
     -2.594418      6.090034      9.971953     12.373220
     -0.891510      6.090034     11.674860     12.373220
      8.327236      6.090034      8.327236     12.373220


2. 导入 _Cordic_ 库中的 _cossin_cordic_ 函数  
cossin_cordic() returns the sine and cosine using the CORDIC method.

In [45]:
Cordic.cossin_cordic(np.pi*1.5, 50)

array([ 1.68995365e-16, -1.00000000e+00])

 ### 自己实现旋转模式下计算$cos\theta$的函数


In [46]:
import numpy as np
def angle_shift(alpha, beta):  # 将alpha 变换到beta~beta+2*pi 且变换后的值不变
    if alpha < beta:
        gamma = beta - np.mod(beta-alpha,2*np.pi) + 2*np.pi
    else:
        gamma = beta + np.mod(alpha-beta,2*np.pi)
    return gamma
#print(angle_shift(5,-np.pi))
#print(angle_shift(1,-np.pi))
#print(angle_shift(-4,-np.pi))
#print(angle_shift(-0.5,-np.pi))

In [47]:
def cordic_cos(theta, n):  #theta角度 n 迭代次数
    theta = angle_shift(theta, -np.pi)

    if theta > np.pi/2:
        flag = -1
        theta -= np.pi
    elif theta < -np.pi/2:
        flag = -1
        theta += np.pi
    else:
        flag = 1       # -pi/2~pi/2方便计算

    v = [1.0, 0.0]  #初始向量
    v_pre = v.copy()
    power2 = 1

    a_i = cal_a_i(n)
    k_list = cal_k(1, n, a_i)
    k = k_list[-1]

    for i in range(n):
        sigma = -1 if theta<0 else 1

        
        v[0] = v_pre[0] - sigma*v_pre[1]*power2
        v[1] = v_pre[1] + sigma*v_pre[0]*power2
        v_pre = v.copy()
        
        theta -= sigma*a_i[i]

        power2 /= 2
    
    v = [i*k for i in v]#模校正因子
    v = [flag*i for i in v]

    return v

In [48]:
print(cordic_cos(-np.pi*1.5,50))
print(cordic_cos(-np.pi/4,50))

[1.689953652959205e-16, 0.9999999999999999]
[0.7071067811865476, -0.7071067811865479]
